# HDS5210-2024 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 24th.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [11]:
def priest(gender, age, resp_rate, oxygen_saturation, heart_rate, systolic_bp, body_temp, alertness, inspired_oxygen, performance_status):
    '''
    (string, float, float, float, float, float, float, string, string, string) -> float
    The function priest computes the Priest score, which measures the risk percentage based on the PRIEST COVID-19 Clinical Severity Score from MDCalc website (https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).

    The function priest includes various input parameters, such as gender, age, respiratory rate, oxygen saturation, heart rate, systolic blood pressure, body temperature, alertness, inspired oxygen assistance, and performance status.

    The result is a risk percentage expressed between 0 and 1 (float). If any of the provided input parameters are invalid, the function returns None.

    >>> priest("Male",18,8.0,96,105.0,115.0,38.1,"ALERT","AIR","limited strenuous activity, can do light activity")
    0.18

    >>> priest("Male",18,12.0,96,115.0,115.0,38.1,"ALERT","AIR","unrestricted normal activity")
    0.03

    >>> priest("FEMALE",40,22.0,93,38.0,95.0,38.1,"Confused or Not Alert","AIR","unrestricted normal activity")
    0.46
    '''

    gender = gender.lower()
    alertness = alertness.lower()
    inspired_oxygen = inspired_oxygen.lower()
    performance_status = performance_status.lower()

    if gender not in ('male', 'female'):
        return None

    if alertness not in ('alert', 'confused or not alert'):
        return None

    if inspired_oxygen not in ('air', 'supplemental oxygen'):
        return None

    valid_statuses = ('unrestricted normal activity', 'limited strenuous activity, can do light activity', 'limited activity, can self-care', 'limited self-care', 'bed/chair bound, no self-care')
    if performance_status not in valid_statuses:
        return None

    count = 0

    if gender == 'male':
        count += 1

    if 50 <= age <= 65:
        count += 2
    elif 66 <= age <= 80:
        count += 3
    elif age > 80:
        count += 4

    if resp_rate < 9:
        count += 3
    elif 9 <= resp_rate <= 11:
        count += 1
    elif 12 <= resp_rate <= 20:
        count += 0
    elif 21 <= resp_rate <= 24:
        count += 2
    elif resp_rate > 24:
        count += 3

    if oxygen_saturation > 95:
        count += 0
    elif 94 <= oxygen_saturation <= 95:
        count += 1
    elif 92 <= oxygen_saturation <= 93:
        count += 2
    elif oxygen_saturation < 92:
        count += 3

    if heart_rate < 41:
        count += 3
    elif 41 <= heart_rate <= 50:
        count += 1
    elif 51 <= heart_rate <= 90:
        count += 0
    elif 91 <= heart_rate <= 110:
        count += 1
    elif 111 <= heart_rate <= 130:
        count += 2
    elif heart_rate > 130:
        count += 3

    if systolic_bp < 91:
        count += 3
    elif 91 <= systolic_bp <= 100:
        count += 2
    elif 101 <= systolic_bp <= 110:
        count += 1
    elif 111 <= systolic_bp <= 219:
        count += 0
    elif systolic_bp > 219:
        count += 3

    if body_temp < 35.1:
        count += 3
    elif 35.1 <= body_temp <= 36:
        count += 1
    elif 36.1 <= body_temp <= 38:
        count += 0
    elif 38.1 <= body_temp <= 39:
        count += 1
    elif body_temp > 39:
        count += 2

    if alertness == "alert":
        count += 0
    elif alertness == "confused or not alert":
        count += 3

    if inspired_oxygen == "air":
        count += 0
    elif inspired_oxygen == "supplemental oxygen":
        count += 2

    performance_status_counts = {
        'unrestricted normal activity': 0,
        'limited strenuous activity, can do light activity': 1,
        'limited activity, can self-care': 2,
        'limited self-care': 3,
        'bed/chair bound, no self-care': 4
    }
    count += performance_status_counts[performance_status]

    if count <= 1:
        return 0.01
    elif 2 <= count <= 3:
        return 0.02
    elif count == 4:
        return 0.03
    elif count == 5:
        return 0.09
    elif count == 6:
        return 0.15
    elif count == 7:
        return 0.18
    elif count == 8:
        return 0.22
    elif count == 9:
        return 0.26
    elif count == 10:
        return 0.29
    elif count == 11:
        return 0.34
    elif count == 12:
        return 0.38
    elif count == 13:
        return 0.46
    elif count == 14:
        return 0.47
    elif count == 15:
        return 0.49
    elif count == 16:
        return 0.55
    elif 17 <= count <= 25:
        return 0.59
    else:
        return 0.99

In [12]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest("Male",18,8.0,96,105.0,115.0,38.1,"ALERT","AIR","limited strenuous activity, can do light activity")
Expecting:
    0.18
ok
Trying:
    priest("Male",18,12.0,96,115.0,115.0,38.1,"ALERT","AIR","unrestricted normal activity")
Expecting:
    0.03
ok
Trying:
    priest("FEMALE",40,22.0,93,38.0,95.0,38.1,"Confused or Not Alert","AIR","unrestricted normal activity")
Expecting:
    0.46
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [19]:
def find_hospital(patient_age, patient_gender, patient_priest_score):
    """
    (int, string, float) -> string

    With patient age, gender, and Priest risk score as inputs, this function outputs the name of a hospital based on data fetched from an API.

    The function sends an API request with patient details and returns the hospital name recommended for the patient. If the API request fails or an exception occurs, the function returns None and prints the error message.

    >>> find_hospital(51, 'FEMALE', 0.29)
    'Select Specialty Hospital - Northeast Atlanta'

    >>> find_hospital(40, 'male', 0.02)
    'Southwest Hospital and Medical Center'

    >>> find_hospital(19, 'MALE', 0.46)
    'Emory Dunwoody Medical Center'
    """
    try:
        # Ensure gender is in lowercase and age is an integer
        patient_gender = patient_gender.lower()
        patient_age = int(patient_age)

        # API endpoint with dynamic patient parameters
        url = f'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={patient_age}&sex={patient_gender}&risk_pct={patient_priest_score}'

        # Perform GET request to the API
        get_response = requests.get(url)

        # Check if request was successful (status code 200)
        if get_response.status_code == 200:
            # Parse the JSON response
            main_json_content = json.loads(get_response.text)

            # Extract the hospital name from the response
            op = main_json_content.get('hospital')

            return op

    except Exception as e:
        # Handle exceptions (e.g., network issues, invalid responses)
        print(f"Error: {e}")
        return None

In [20]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(51, 'FEMALE', 0.29)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(40, 'male', 0.02)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(19, 'MALE', 0.46)
Expecting:
    'Emory Dunwoody Medical Center'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [21]:
import requests
import json

def get_address(name):
    """
    (string) -> string

    This function fetches the hospital's address using the hospital name as the input. It sends a request to a Google Drive-hosted JSON file that contains hospital data, including addresses. If the hospital name is found, it returns the corresponding address. If the name is not found or an error occurs, the function returns None.

    >>> get_address('Select Specialty Hospital - Northeast Atlanta')
    '1821 CLIFTON ROAD NE'

    >>> get_address('Southwest Hospital and Medical Center')
    '501 FAIRBURN ROAD SW'

    >>> get_address('Emory Dunwoody Medical Center')
    '4500 NORTH SHALLOWFORD ROAD'
    """
    try:
        # Convert the hospital name to uppercase
        name = name.upper()

        # URL to fetch the JSON data of hospitals and addresses
        address_response_url = 'https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-'

        # Perform the GET request to fetch the data
        get_response = requests.get(address_response_url)

        # Check if the request was successful
        if get_response.status_code == 200:
            # Parse the JSON response
            main_address_data = json.loads(get_response.text)

            # Extract the hospital data using the hospital name
            hospital_data = main_address_data.get(name)

            # If hospital data is found, return the address
            if hospital_data:
                return hospital_data.get('ADDRESS')

    except Exception as e:
        # Handle exceptions (e.g., network issues, invalid data)
        print(f"Error: {e}")

    return None

# Running doctests
import doctest
doctest.run_docstring_examples(get_address, globals(), verbose=True)

Finding tests in NoName
Trying:
    get_address('Select Specialty Hospital - Northeast Atlanta')
Expecting:
    '1821 CLIFTON ROAD NE'
ok
Trying:
    get_address('Southwest Hospital and Medical Center')
Expecting:
    '501 FAIRBURN ROAD SW'
ok
Trying:
    get_address('Emory Dunwoody Medical Center')
Expecting:
    '4500 NORTH SHALLOWFORD ROAD'
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [22]:
import requests

def process_people(link):
    """
    (string) -> dict

    This function extracts patient information from a specified URL, applies multiple
    processing steps, and generates a dictionary containing patient specifics.

    Each patient is distinguished by a unique patient ID, which serves as the dictionary key.

    The function fetches patient data from the link, processes each patient's information,
    calculates their PRIEST score, and finds the hospital name and address.

    Returns:
        dict: A dictionary where each key is a patient's unique ID, and the value is a list
        containing patient specifics such as gender, age, vitals, PRIEST score, hospital name,
        and address.
    """
    try:
        # Fetching data from the provided link
        get_response = requests.get(link)
        if get_response.status_code != 200:
            raise Exception("Failed to fetch patient data.")

        # Decode content and process each line (skip the first line as it's the header)
        content = get_response.content
        data_list = content.decode().split('\n')[1:]

        op_dict = {}  # Dictionary to store processed data

        for row in data_list:
            # Split each row by the '|' delimiter and strip spaces
            row_list = row.split('|')
            if len(row_list) == 11:
                (
                    patient_id, patient_gender, patient_age, patient_res_rate, patient_o2_sat,
                    patient_hr, patient_sys_bp, patient_temperature, patient_alert, patient_ins_o2, patient_p_status
                ) = map(str.strip, row_list)

                # Convert string inputs to appropriate types
                patient_age = int(patient_age)
                patient_res_rate = int(patient_res_rate)
                patient_o2_sat = float(patient_o2_sat) * 100
                patient_hr = int(patient_hr)
                patient_sys_bp = int(patient_sys_bp)
                patient_temperature = float(patient_temperature)

                # Apply the priest() function to calculate the PRIEST score
                priest_score = priest(
                    patient_gender, patient_age, patient_res_rate, patient_o2_sat,
                    patient_hr, patient_sys_bp, patient_temperature, patient_alert.strip(),
                    patient_ins_o2.strip(), patient_p_status.strip()
                )

                # Determine the hospital name and address
                hospital_name = find_hospital(patient_age, patient_gender, priest_score)
                hospital_address = get_address(hospital_name)

                # Store patient data in the dictionary
                op_dict[patient_id] = [
                    patient_gender, patient_age, patient_res_rate, patient_o2_sat / 100,
                    patient_hr, patient_sys_bp, patient_temperature, patient_alert,
                    patient_ins_o2, patient_p_status, priest_score, hospital_name, hospital_address
                ]

        return op_dict

    except Exception as e:
        # Error handling
        print(f"Error: {e}")
        return None

# Process the patient data from the provided link
main_patient_data = process_people('https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh')

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [23]:
import requests
import json

def fetch_data_from_url(link):
    """
   This function fetches data from a given URL and returns it as a Python dictionary.
    """
    get_response = requests.get(link)

    if get_response.status_code != 200:
        return None

    content = get_response.content
    return json.loads(content)

def compare_dictionary_values(data1, data2):
    """
   This function compares two dictionaries of patient data and identifies any differences.
    """
    errors = []

    for patient_id, data_list in data1.items():

        data_list_2 = data2.get(patient_id)

        if data_list_2:

            for data_index in range(10, 13):

                if data_list[data_index] != data_list_2[data_index]:
                    errors.append((patient_id, data_index))

    return errors

given_url = 'https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5'

obtained_data = fetch_data_from_url(given_url)

if obtained_data is not None:

    data_differences = compare_dictionary_values(main_patient_data, obtained_data)

    if data_differences:

        for patient_id, index in data_differences:
            print(f"Error at: {patient_id}  index: {index}")

    else:
        print("Successfully matched")

else:
    print("Failed to fetch data from the given URL.")

Successfully matched


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---